In [ ]:
import pandas as pd
import random
import os
import requests
import csv
import time

# --- Cấu hình Moodle API ---
MOODLE_URL = 'http://localhost:8100/webservice/rest/server.php'
TOKEN = '84cffdbb9ead18d97ccc45f9889bc926'
FORMAT = 'json'

# --- Actions ---
ACTIONS = [
    'read_new_resource',
    'review_old_resource',
    'skip_to_next_module',
    'attempt_easier_quiz',
    'attempt_harder_quiz',
    'redo_similar_quiz',
    'return_previous_module'
]

def call_api(function_name, params):
    params.update({
        'wstoken': TOKEN,
        'wsfunction': function_name,
        'moodlewsrestformat': FORMAT
    })
    try:
        response = requests.get(MOODLE_URL, params=params)
        response.raise_for_status()
        return response.json()
    except Exception as e:
        print(f"❌ API {function_name} lỗi: {e}")
        return {}

def get_user_cluster(user_id):
    """
    Lấy thông tin cluster của người dùng từ file CSV.
    """
    file_path = "./synthetic_user_features_clustered.csv"
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print("❌ Không tìm thấy file: synthetic_user_features_clustered.csv")
        return None
    user_row = df[df['userid'] == user_id]
    if user_row.empty:
        return None
    return user_row.iloc[0]['cluster']

def track_and_update_new_rows(log_file_path, poll_interval=2):
    print(f"\n🔍 Đang theo dõi file log: {log_file_path}")
    seen_lines = 0
    output_path = './user_insight.csv'

    if not os.path.exists(output_path):
        with open(output_path, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([
                'userid', 'courseid', 'sectionid', 'type',
                'avg_quiz_score', 'completion_rate', 'quiz_passed',
                'cluster', 'time'
            ])

    while True:
        try:
            if not os.path.exists(log_file_path):
                print("⚠️ File log chưa tồn tại. Đợi...")
                time.sleep(poll_interval)
                continue
            
            df_log = pd.read_csv(log_file_path)
            current_lines = len(df_log)

            if current_lines > seen_lines:
                new_rows = df_log.iloc[seen_lines:]
                print(f"\n🆕 Phát hiện {len(new_rows)} dòng log mới:")
                seen_lines = current_lines

                for _, row in new_rows.iterrows():
                    userid = int(row['userid'])
                    courseid = int(row['courseid'])
                    sectionid = int(row['sectionid'])
                    typ = str(row['type'])
                    timestamp = int(row['time'])

                    print(f"\n--- Xử lý User {userid} ({typ}) tại Section {sectionid} ---")

                    avg_quiz_score = call_api('local_userlog_get_avg_quiz_score', {
                        'userid': userid, 'courseid': courseid
                    }).get('avg_quiz_score', 0.0)

                    total_resource = call_api('local_userlog_get_total_resources_by_section', {
                        'sectionid': sectionid,
                        'objecttypes[0]': 'resource',
                        'objecttypes[1]': 'hvp',
                    }).get('total_resources', 0)

                    viewed_resource = call_api('local_userlog_get_viewed_resources_distinct_by_section', {
                        'userid': userid,
                        'courseid': courseid,
                        'sectionid': sectionid,
                        'objecttypes[0]': 'resource',
                        'objecttypes[1]': 'hvp',
                    }).get('viewed_resources', 0)

                    completion_rate = viewed_resource / total_resource if total_resource > 0 else 0

                    quiz_passed = call_api('local_userlog_get_latest_quiz_pass_status_by_section', {
                        'sectionid': sectionid,
                        'userid': userid,
                    }).get('is_passed', 0)

                    cluster = get_user_cluster(userid)

                    print(f"[DEBUG] cluster={cluster}, "
                          f"avg_quiz_score={avg_quiz_score}, "
                          f"completion_rate={completion_rate:.2f}, "
                          f"quiz_passed={quiz_passed}")

                    with open(output_path, 'a', newline='') as f:
                        writer = csv.writer(f)
                        writer.writerow([
                            userid,
                            courseid,
                            sectionid,
                            typ,
                            avg_quiz_score,
                            round(completion_rate, 2),
                            int(quiz_passed),
                            int(cluster) if pd.notna(cluster) else 0,
                            timestamp
                        ])

            time.sleep(poll_interval)

        except pd.errors.EmptyDataError:
            print("⚠️ File log rỗng. Đợi dữ liệu mới...")
            time.sleep(poll_interval)
        except Exception as e:
            print(f"❌ Lỗi: {e}")
            time.sleep(poll_interval)

if __name__ == "__main__":
    log_file_path = "/Users/nguyenhuuloc/Documents/MyComputer/moodledata/local_userlog_data/user_log_summary.csv"
    track_and_update_new_rows(log_file_path)


🔍 Đang theo dõi file log: /Users/nguyenhuuloc/Documents/MyComputer/moodledata/local_userlog_data/user_log_summary.csv

🆕 Phát hiện 223 dòng log mới:

--- Xử lý User 4 (quiz) tại Section 42 ---
[DEBUG] cluster=0.0, avg_quiz_score=10, completion_rate=1.00, quiz_passed=1

--- Xử lý User 4 (resource) tại Section 38 ---
[DEBUG] cluster=0.0, avg_quiz_score=10, completion_rate=1.00, quiz_passed=0

--- Xử lý User 4 (hvp) tại Section 38 ---
[DEBUG] cluster=0.0, avg_quiz_score=10, completion_rate=1.00, quiz_passed=0

--- Xử lý User 4 (quiz) tại Section 42 ---
[DEBUG] cluster=0.0, avg_quiz_score=10, completion_rate=1.00, quiz_passed=1

--- Xử lý User 4 (quiz) tại Section 42 ---
[DEBUG] cluster=0.0, avg_quiz_score=10, completion_rate=1.00, quiz_passed=1

--- Xử lý User 4 (quiz) tại Section 42 ---
[DEBUG] cluster=0.0, avg_quiz_score=10, completion_rate=1.00, quiz_passed=1

--- Xử lý User 4 (hvp) tại Section 38 ---
[DEBUG] cluster=0.0, avg_quiz_score=10, completion_rate=1.00, quiz_passed=0

--- Xử 